## Метод CatBoost

### Лабораторная работа №6

#### Задание 1
Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

Гиперпараметры:
   - depth: Этот параметр определяет глубину дерева.
   - learning_rate: Определяет скорость обучения модели.
   - l2_leaf_reg: Этот параметр отвечает за коэффициент регуляризации L2 (Ridge).

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm

# Загрузка датасета
file_path = "Electric_Vehicle_Population_Data.csv"
df = pd.read_csv(file_path)
df.dropna(inplace=True)
label_encoder = LabelEncoder()
df['Electric Vehicle Type'] = label_encoder.fit_transform(df['Electric Vehicle Type'])
df = df.sample(n=10000)

# Разделение на признаки (X) и целевую переменную (y)
X = df[['Postal Code', 'Model Year']]
y = df['Electric Vehicle Type']

# Разделение данных на тренировочный и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# Определение параметров для CatBoost
param_grid_catboost = {
    'depth': [1, 4, 7, 10],
    'learning_rate': [0.01, 0.1, 1],
    'l2_leaf_reg': [1, 3, 5, 9],
    'iterations': [100, 200,],
    'depth': [0, 3, 6],
    'loss_function': ['MultiClass', 'Logloss']
}

# Создание модели CatBoost
catboost = CatBoostClassifier(verbose=0)

# Подбор оптимальных параметров с помощью перекрестной проверки для CatBoost
grid_search_catboost = GridSearchCV(catboost, param_grid_catboost, refit=True, verbose=0, cv=5)
grid_search_catboost.fit(X_train_scaled, y_train)

# Получение лучших параметров для CatBoost
best_params_catboost = grid_search_catboost.best_params_
best_score_catboost = grid_search_catboost.best_score_
test_score = grid_search_catboost.score(X_test_scaled, y_test)
print("Лучшие параметры для CatBoost:", best_params_catboost)
print("Лучший результат (точность) для CatBoost:", best_score_catboost)
print("Точность на тестовом наборе данных:", test_score)

best_model = grid_search_catboost.best_estimator_
predictions = best_model.predict(X_test_scaled)

# Отчет по классификации
print("\nОтчет по классификации:")
print(classification_report(y_test, predictions))

Лучшие параметры для CatBoost: {'depth': 3, 'iterations': 200, 'l2_leaf_reg': 1, 'learning_rate': 0.1, 'loss_function': 'Logloss'}
Лучший результат (точность) для CatBoost: 0.7897777777777778
Точность на тестовом наборе данных: 0.778

Отчет по классификации:
              precision    recall  f1-score   support

           0       0.79      0.97      0.87       760
           1       0.66      0.16      0.26       240

    accuracy                           0.78      1000
   macro avg       0.72      0.57      0.56      1000
weighted avg       0.75      0.78      0.72      1000

